In [1]:
import json
import os
import re

In [2]:
import cloudscraper
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd
import h3

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Dunkin Data

In [6]:
scraper = cloudscraper.create_scraper()

In [7]:
def get_coords(store_url: str) -> Point:
    r = scraper.get(store_url)
    soup = BeautifulSoup(r.text, "html.parser")
    lat = float(
        re.search(r'"latitude"\s*:\s*([-]?\d+\.\d+)', soup.find("script").text).group(1)
    )
    lon = float(
        re.search(r'"longitude"\s*:\s*([-]?\d+\.\d+)', soup.find("script").text).group(
            1
        )
    )
    return Point(lon, lat)

In [17]:
url_root = "https://locations.dunkindonuts.com"

# Parse all locations in a state
for i in tqdm(range(len(states_df)), desc="Parsing States"):
    state_store_list = []
    state_code = states_df.iloc[i]["STUSPS"].lower()

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    state_url = os.path.join(url_root, "en/" + state_code)
    locations_r = scraper.get(state_url)

    if locations_r.status_code != 200:
        print(state_code)
        continue

    soup_state = BeautifulSoup(locations_r.text, "html.parser")
    city_as = soup_state.find_all(
        "a",
        {
            "class": "Link no-underline inline-flex p-4 sm:p-0 justify-between items-center w-full"
        },
    )

    for city_a in tqdm(city_as, desc=f"Parsing Locations in {state_code}"):
        href = city_a.attrs["href"]
        count = int(re.findall(r"\d+", city_a.find("span").attrs["data-count"])[0])

        if count == 1:
            store_url = href.replace("..", url_root)

            try:
                point = get_coords(store_url)
                store_dict = {"STUSPS": state_code.upper(), "geometry": point}
                state_store_list.append(store_dict)
            except Exception as e:
                print(e, store_url)

            continue

        city_url = href.replace("..", url_root)
        r_city = scraper.get(city_url)
        soup_city = BeautifulSoup(r_city.text, "html.parser")
        store_as = soup_city.find_all(
            "a",
            {
                "class": "Link no-underline border border-brand-gray-200 hover:border-brand-secondary rounded-[16px] p-8 sm:p-10 h-full block"
            },
        )

        for store_a in store_as:
            store_url = store_a.attrs["href"].replace("../..", url_root)
            try:
                point = get_coords(store_url)
                store_dict = {"STUSPS": state_code.upper(), "geometry": point}
                state_store_list.append(store_dict)
            except Exception as e:
                print(e, store_url)

    if state_store_list:
        print(len(state_store_list), "stores")
        state_dunkin_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        state_dunkin_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/56 [00:00<?, ?it/s]

Parsing Locations in nc:   0%|          | 0/111 [00:00<?, ?it/s]

229 stores


Parsing Locations in ok:   0%|          | 0/13 [00:00<?, ?it/s]

23 stores


...

'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/in/hammond/7935-indianapolis-boulevard/358736
124 stores


Parsing Locations in mn:   0%|          | 0/35 [00:00<?, ?it/s]

'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/mn/duluth/104-west-central-entrance/353944
41 stores


Parsing Locations in ct:   0%|          | 0/157 [00:00<?, ?it/s]

'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/bristol/1053-farmington-ave/345665
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/madison/720-route-95-north/349431
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/manchester/1205-tolland-tpke/306504
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/manchester/318-adams-st/343320
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/manchester/81-oakland-st/310194
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/orange/3-wilbur-cross-parkway-nb/349429
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/plainfield/rt-395-southbound/349415
'NoneType' object has no attribute 'group' https://locations.dunkindonuts.com/en/ct/stamford/30-station-pl/336890
'NoneType' object has no attribute 'group' https://loc

In [24]:
added_dunkins = [
    {"STUSPS": "CT", "geometry": Point(41.754286501902484, -71.87852045766724)},
    {"STUSPS": "CT", "geometry": Point(41.04674704030949, -73.54207345763879)},
    {"STUSPS": "CT", "geometry": Point(41.72391345834193, -72.76283231532206)},
    {"STUSPS": "CT", "geometry": Point(41.92995211964665, -72.68351842466396)},
    {"STUSPS": "CT", "geometry": Point(41.929732710831, -72.684471757736)},
    {"STUSPS": "IN", "geometry": Point(41.569458440202226, -87.47499044418842)},
    {"STUSPS": "MN", "geometry": Point(46.80013730594761, -92.13581944413997)},
    {"STUSPS": "AR", "geometry": Point(36.262820230178384, -93.13454364421959)},
    {"STUSPS": "AR", "geometry": Point(35.85595143908745, -90.660867113483)},
    {"STUSPS": "OH", "geometry": Point(41.41091286966159, -81.83890696929409)},
    {"STUSPS": "OH", "geometry": Point(41.36782623309512, -82.9587364404243)},
    {"STUSPS": "ME", "geometry": Point(43.458161306067765, -70.79234869815791)},
    {"STUSPS": "ME", "geometry": Point(44.19779937833875, -70.2436976288386)},
    {"STUSPS": "MD", "geometry": Point(39.497434307984925, -76.23218037297032)},
    {"STUSPS": "MD", "geometry": Point(38.32824660812148, -75.61029249628726)},
    {"STUSPS": "MD", "geometry": Point(39.624718785586445, -77.6737266693742)},
    {"STUSPS": "NJ", "geometry": Point(40.72340826552654, -74.1790485000014)},
    {"STUSPS": "NJ", "geometry": Point(40.71885925024087, -74.16111118655158)},
    {"STUSPS": "NJ", "geometry": Point(40.684081864048274, -74.18797492431557)},
    {"STUSPS": "NJ", "geometry": Point(40.820309702021305, -74.10040532884034)},
    {"STUSPS": "NJ", "geometry": Point(40.38595950718633, -74.44448270584272)},
    {"STUSPS": "NJ", "geometry": Point(40.101610114435935, -74.7966029711416)},
    {"STUSPS": "NJ", "geometry": Point(40.22622691708618, -74.66427638659187)},
    {"STUSPS": "NJ", "geometry": Point(40.927330983349115, -74.95854256735116)},
    {"STUSPS": "NJ", "geometry": Point(40.262680889960166, -74.43286839040037)},
    {"STUSPS": "NJ", "geometry": Point(40.38217980755236, -74.51807312712059)},
    {"STUSPS": "NJ", "geometry": Point(40.933415957983776, -74.11850104419173)},
    {"STUSPS": "NJ", "geometry": Point(39.77208439217998, -74.9001633692162)},
    {"STUSPS": "NJ", "geometry": Point(39.7171170412217, -74.90014775763805)},
    {"STUSPS": "NJ", "geometry": Point(39.450561087305054, -74.57166453469667)},
    {"STUSPS": "NJ", "geometry": Point(40.88150978063889, -73.98632808472692)},
    {"STUSPS": "DC", "geometry": Point(38.899748121269376, -77.03912117502784)},
    {"STUSPS": "VA", "geometry": Point(38.8721783939383, -77.05618108271511)},
    {"STUSPS": "AZ", "geometry": Point(33.27545989415952, -111.8579851441912)},
    {"STUSPS": "AZ", "geometry": Point(33.32078843157168, -111.72062358826375)},
    {"STUSPS": "AZ", "geometry": Point(34.72826701487247, -114.3156597170695)},
    {"STUSPS": "VT", "geometry": Point(43.404336850051195, -72.41123072881537)},
    {"STUSPS": "NH", "geometry": Point(43.24360373075477, -71.40508070000098)},
    {"STUSPS": "NH", "geometry": Point(42.77010934493543, -71.4917666577028)},
    {"STUSPS": "IL", "geometry": Point(42.46872118652512, -88.06455633872041)},
    {"STUSPS": "IL", "geometry": Point(41.782495997987226, -88.23997885586925)},
    {"STUSPS": "IL", "geometry": Point(41.71130392217927, -88.25358063237103)},
    {"STUSPS": "IL", "geometry": Point(41.890711466886444, -87.66664325767968)},
    {"STUSPS": "IL", "geometry": Point(41.876771590661676, -87.63341984812915)},
    {"STUSPS": "IL", "geometry": Point(41.97513888325788, -87.88998068285552)},
    {"STUSPS": "IL", "geometry": Point(41.68103687717942, -88.20391794764383)},
    {"STUSPS": "IL", "geometry": Point(41.65344883734468, -87.8531498865651)},
    {"STUSPS": "IL", "geometry": Point(42.04828264433846, -88.09105101355834)},
    {"STUSPS": "NV", "geometry": Point(36.12948346384014, -115.15446778650971)},
    {"STUSPS": "NY", "geometry": Point(40.71352306286673, -73.87392219811173)},
    {"STUSPS": "NY", "geometry": Point(40.70229620129257, -73.81684705579092)},
    {"STUSPS": "NY", "geometry": Point(40.624632843899086, -74.14815075959103)},
    {"STUSPS": "NY", "geometry": Point(40.64750551200695, -73.79067714421964)},
    {"STUSPS": "NY", "geometry": Point(40.6436657293237, -73.7905555103908)},
    {"STUSPS": "NY", "geometry": Point(40.643023556421625, -73.7903647957481)},
    {"STUSPS": "NY", "geometry": Point(40.82538235666321, -72.98213348656833)},
    {"STUSPS": "NY", "geometry": Point(40.75174425348389, -73.97737961350093)},
    {"STUSPS": "NY", "geometry": Point(42.74773290321274, -76.13538180182165)},
    {"STUSPS": "NY", "geometry": Point(40.76538438520437, -73.86205655948531)},
    {"STUSPS": "NY", "geometry": Point(40.70247913224715, -73.79185101897774)},
    {"STUSPS": "NY", "geometry": Point(40.642986620554225, -73.78366465187679)},
    {"STUSPS": "NY", "geometry": Point(40.64303683359327, -73.7906509654871)},
    {"STUSPS": "NY", "geometry": Point(40.64757791062497, -73.79031640251786)},
    {"STUSPS": "NY", "geometry": Point(40.849235879611996, -73.88347374233237)},
    {"STUSPS": "NY", "geometry": Point(40.86017401438138, -73.90258613073937)},
    {"STUSPS": "NY", "geometry": Point(40.60679175574763, -73.96279270179576)},
    {"STUSPS": "NY", "geometry": Point(42.909613764264954, -73.8994144711684)},
    {"STUSPS": "NY", "geometry": Point(43.168189737602546, -76.51136099999137)},
    {"STUSPS": "MO", "geometry": Point(37.56730971833378, -90.32297039104101)},
    {"STUSPS": "MO", "geometry": Point(38.62323304193598, -90.33362116944677)},
    {"STUSPS": "CO", "geometry": Point(38.754130549024076, -104.7871115423505)},
    {"STUSPS": "CO", "geometry": Point(39.05575484045122, -104.85376034232999)},
    {"STUSPS": "PA", "geometry": Point(40.6523338712971, -75.43560064234464)},
    {"STUSPS": "PA", "geometry": Point(39.92552511890636, -77.61671456920033)},
    {"STUSPS": "PA", "geometry": Point(40.495774603032444, -80.25617791533756)},
    {"STUSPS": "PA", "geometry": Point(40.496007374307304, -80.24568206113817)},
    {"STUSPS": "PA", "geometry": Point(40.2818360144692, -74.99483725765916)},
    {"STUSPS": "PA", "geometry": Point(40.38154114099451, -75.14669068467698)},
    {"STUSPS": "PA", "geometry": Point(40.21923332701709, -75.12097424237238)},
    {"STUSPS": "PA", "geometry": Point(40.21496236060761, -80.19962311346772)},
    {"STUSPS": "GA", "geometry": Point(33.64094291158585, -84.44615694160487)},
    {"STUSPS": "GA", "geometry": Point(33.64229437461597, -84.43584252883242)},
    {"STUSPS": "GA", "geometry": Point(33.56037799134848, -84.32225171347994)},
    {"STUSPS": "GA", "geometry": Point(32.99454369623157, -85.02468121722868)},
    {"STUSPS": "GA", "geometry": Point(31.448067135748474, -83.53844165413472)},
    {"STUSPS": "CA", "geometry": Point(33.21753241496846, -87.54619678464068)},
    {"STUSPS": "AL", "geometry": Point(34.03684223401944, -118.44415632743608)},
    {"STUSPS": "TX", "geometry": Point(32.84567142486981, -96.84991764415028)},
    {"STUSPS": "TX", "geometry": Point(32.84562635656773, -96.8498532711796)},
    {"STUSPS": "TX", "geometry": Point(31.792850669365706, -106.26035368654158)},
    {"STUSPS": "TX", "geometry": Point(33.218530690217825, -96.87014687304624)},
    {"STUSPS": "TX", "geometry": Point(32.80184383661128, -97.03044984416309)},
    {"STUSPS": "TX", "geometry": Point(31.843537011575712, -103.10387845351553)},
    {"STUSPS": "TX", "geometry": Point(31.853429199078445, -103.06764601589605)},
    {"STUSPS": "TX", "geometry": Point(33.202576156885904, -96.73384144416352)},
    {"STUSPS": "TX", "geometry": Point(31.5875320073589, -102.91746945908615)},
    {"STUSPS": "TX", "geometry": Point(30.086718041665637, -95.989460715347)},
    {"STUSPS": "FL", "geometry": Point(30.194993309695413, -81.82508198651242)},
    {"STUSPS": "FL", "geometry": Point(30.391101472088078, -81.4152165711754)},
    {"STUSPS": "FL", "geometry": Point(26.59248634176106, -80.20673861538502)},
    {"STUSPS": "FL", "geometry": Point(26.226113441391103, -80.18322903426454)},
    {"STUSPS": "FL", "geometry": Point(25.79408090816204, -80.27823619806102)},
    {"STUSPS": "FL", "geometry": Point(25.794617511250493, -80.27915899999887)},
    {"STUSPS": "FL", "geometry": Point(29.26632029228428, -82.19411128460669)},
    {"STUSPS": "FL", "geometry": Point(29.829902531676584, -81.376239700004)},
    {"STUSPS": "MA", "geometry": Point(41.713695457570395, -70.48949014230323)},
    {"STUSPS": "MA", "geometry": Point(42.10234909940595, -72.59262191532598)},
    {"STUSPS": "MA", "geometry": Point(42.62139704212644, -71.18471634405807)},
    {"STUSPS": "MA", "geometry": Point(42.085995905715194, -71.6953464981217)},
    {"STUSPS": "MA", "geometry": Point(41.78903280298809, -70.7505340288379)},
    {"STUSPS": "MA", "geometry": Point(42.24530562602001, -71.59204106784708)},
    {"STUSPS": "MA", "geometry": Point(42.567046854024, -71.42377952509118)},
    {"STUSPS": "MA", "geometry": Point(42.36635292846306, -71.0940975576558)},
    {"STUSPS": "MA", "geometry": Point(41.682186478721306, -69.99146264411009)},
    {"STUSPS": "MA", "geometry": Point(42.539766762752514, -71.61400079999423)},
    {"STUSPS": "MA", "geometry": Point(42.36665919354714, -71.0166829780522)},
    {"STUSPS": "MA", "geometry": Point(41.681342726252026, -71.13194270000571)},
    {"STUSPS": "VA", "geometry": Point(38.753554247872586, -77.1970829559064)},
    {"STUSPS": "VA", "geometry": Point(36.915414811009704, -76.21815824417234)},
    {"STUSPS": "WV", "geometry": Point(38.41536907150278, -82.25813075585062)},
    {"STUSPS": "LA", "geometry": Point(30.177286139033633, -93.12980565585583)},
    {"STUSPS": "MI", "geometry": Point(44.73888705682888, -85.64763225191356)},
    {"STUSPS": "MI", "geometry": Point(42.47074507072462, -83.28026474232622)},
    {"STUSPS": "MA", "geometry": Point(42.3458784112385, -71.09886448142649)},
    {"STUSPS": "MA", "geometry": Point(42.36436721561039, -71.06292885842072)},
    {"STUSPS": "MA", "geometry": Point(42.366308052557116, -71.01738044450374)},
    {"STUSPS": "MA", "geometry": Point(42.36631040682103, -71.01688683775811)},
    {"STUSPS": "MA", "geometry": Point(41.75710837750203, -70.65354744771972)},
]

In [36]:
added_dunkins = gpd.GeoDataFrame(added_dunkins)
added_dunkins["geometry"] = added_dunkins.apply(
    lambda row: Point(row["geometry"].y, row["geometry"].x), axis=1
)
added_dunkins = added_dunkins.set_crs(4326)

In [37]:
dunkin_state_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        state_dunkin_gdf = gpd.read_file(constructed_path)
        dunkin_state_gdfs.append(state_dunkin_gdf)
dunkin_state_gdfs.append(added_dunkins)

In [38]:
dunkin_gdf = gpd.GeoDataFrame(pd.concat(dunkin_state_gdfs, ignore_index=True))

In [46]:
dunkin_gdf = dunkin_gdf.to_crs(9311)
dunkin_gdf.to_file(f"data/dunkins.gpkg")

## Get Population Data

In [40]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [41]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [42]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [43]:
states_with_population_df = states_with_population_df.dropna()

## Merge Data

In [52]:
dunkin_state_counts_df = pd.DataFrame(
    dunkin_gdf.groupby("STUSPS").size(), columns=["DUNKINS"]
).reset_index()

In [53]:
dunkin_per_state_gdf = states_with_population_df.merge(
    dunkin_state_counts_df, on="STUSPS", how="left"
)
dunkin_per_state_gdf = dunkin_per_state_gdf.fillna(0)

In [54]:
dunkin_per_state_gdf["per_1000"] = dunkin_per_state_gdf["DUNKINS"] / (
    dunkin_per_state_gdf["POPULATION"] / 1000
)
dunkin_per_state_gdf["per_10k"] = dunkin_per_state_gdf["DUNKINS"] / (
    dunkin_per_state_gdf["POPULATION"] / 10_000
)
dunkin_per_state_gdf["per_100k"] = dunkin_per_state_gdf["DUNKINS"] / (
    dunkin_per_state_gdf["POPULATION"] / 100000
)
dunkin_per_state_gdf["per_500k"] = dunkin_per_state_gdf["DUNKINS"] / (
    dunkin_per_state_gdf["POPULATION"] / 500_000
)
dunkin_per_state_gdf["per_1m"] = dunkin_per_state_gdf["DUNKINS"] / (
    dunkin_per_state_gdf["POPULATION"] / 1_000_000
)
dunkin_per_state_gdf["per_capita"] = (
    dunkin_per_state_gdf["POPULATION"] / dunkin_per_state_gdf["DUNKINS"]
)

In [55]:
dunkin_per_state_gdf = dunkin_per_state_gdf.to_crs(9311)

In [56]:
dunkin_per_state_gdf.to_file("data/dunkins_per_state.gpkg")